In [ ]:
import pandas as pd
import numpy as np
import copy, time
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns # 另一個繪圖-樣式套件

########## load data start
data_path = '../csv/3rd-ml100marathon-midterm/'
df_train_tmp = pd.read_csv(data_path + 'train_data.csv')
df_test = pd.read_csv(data_path + 'test_features.csv')

########## load data start end


# 檢查 DataFrame 空缺值的狀態
def na_check(df_data):
    data_na = (df_data.isnull().sum() / len(df_data)) * 100
    data_na = data_na.drop(data_na[data_na == 0].index).sort_values(ascending=False)
    missing_data = pd.DataFrame({'Missing Ratio' :data_na})
    display(missing_data.head(10))
    
def plot(plot_datas):
    plot_datas_new = plot_datas.copy()
    plt.figure(figsize = (24, 8))
    feat_labels = plot_datas_new.columns
    print('feat_labels ', feat_labels)
# 依不同 EXT_SOURCE 逐項繪製 KDE 圖形
    for i, source in enumerate(feat_labels):
        if source == 'poi':
            continue
            
        source_array = [source,'poi']
        var_plot_data = plot_datas_new[source_array]
        print('######################################################################')
        print('describe : ', var_plot_data[source].describe())
        var_plot_data = var_plot_data.dropna()

        q25, q75 = np.percentile(var_plot_data[source], 25), np.percentile(var_plot_data[source], 75)
        iqr = q75 - q25
        print('Percentiles: 25th=%.3f, 75th=%.3f, IQR=%.3f' % (q25, q75, iqr))
        # calculate the outlier cutoff
        cut_off = iqr * 3
        lower, upper = q25 - cut_off, q75 + cut_off
        # identify outliers
        outliers = [x for x in var_plot_data[source] if x < lower or x > upper]
        print('Identified outliers: %d' % len(outliers))
        # remove outliers
        outliers_removed = [x for x in var_plot_data[source] if x >= lower and x <= upper]
        print('Non-outlier observations: %d' % len(outliers_removed))
        print('feat_label ', source)
        # KDE 圖形
        sns.kdeplot(var_plot_data.loc[var_plot_data['poi'] == False, source], label = 'poi == 0')
        sns.kdeplot(var_plot_data.loc[var_plot_data['poi'] == True, source], label = 'poi == 1')
        
        # 加上各式圖形標籤
        plt.title('Distribution of %s by Target Value' % source)
        plt.xlabel('%s' % source); plt.ylabel('Density');
        plt.show()
        #######
        var_plot_data_outlier_removed = var_plot_data.loc[(var_plot_data[source]>= lower) & (var_plot_data[source] <= upper)]
        sns.kdeplot(var_plot_data_outlier_removed.loc[var_plot_data_outlier_removed['poi'] == False, source], label = 'poi == 0')
        sns.kdeplot(var_plot_data_outlier_removed.loc[var_plot_data_outlier_removed['poi'] == True, source], label = 'poi == 1')
        
        # 加上各式圖形標籤
        plt.title('Distribution of %s by Target Value' % source)
        plt.xlabel('%s' % source); plt.ylabel('Density');
        plt.show()
        
        ########
        var_plot_data_2 = var_plot_data
        var_plot_data_2[source] = var_plot_data[source].clip(lower, upper)
        sns.kdeplot(var_plot_data_2.loc[var_plot_data_2['poi'] == False, source], label = 'poi == 0')
        sns.kdeplot(var_plot_data_2.loc[var_plot_data_2['poi'] == True, source], label = 'poi == 1')
        
        # 加上各式圖形標籤
        plt.title('Distribution of %s by Target Value' % source)
        plt.xlabel('%s' % source); plt.ylabel('Density');
        plt.show()


########## 前處理理 (Processing) start
#薪水
var_salary = ['salary']

#獎金
var_bonus = ['bonus']

#業務報銷
var_business_expenses_group = ['long_term_incentive','deferred_income','loan_advances'
        ,'deferral_payments','other','expenses']

#董事费
var_director_fees = ['director_fees']

#總支出
var_total_payments = ['total_payments']

#股權
var_10_11_12 = ['exercised_stock_options','restricted_stock','restricted_stock_deferred']

#股權 total
var_total_stock_value = ['total_stock_value']

#email回覆
var_email_count_group = ['to_messages','from_poi_to_this_person','from_messages','from_this_person_to_poi','shared_receipt_with_poi']


df_train_tmp = df_train_tmp.drop(['name', 'email_address'] , axis=1)

df_train_tmp = df_train_tmp.drop(['director_fees'] , axis=1)
df_train_tmp = df_train_tmp.drop(['restricted_stock_deferred'] , axis=1)
df_train_tmp = df_train_tmp.drop(['loan_advances'] , axis=1)


feat_labels = df_train_tmp.columns
feat_labels

########## 前處理理 (Processing) end



In [ ]:
ext_data_corrs = df_train_tmp.corr()

threshold = 0.2
threshold2 = -1*threshold
ext_data_corrs = ext_data_corrs.loc[:, ['poi']]

ext_data_corrs = ext_data_corrs.loc[(ext_data_corrs['poi']>threshold) | (ext_data_corrs['poi']<threshold2)]
ext_data_corrs_index = ext_data_corrs.index

plt.figure(figsize = (8, 6))
# 繪製相關係數 (correlations) 的 Heatmap
sns.heatmap(ext_data_corrs, cmap = plt.cm.RdYlBu_r, vmin = -0.25, annot = True, vmax = 0.6)
plt.title('Correlation Heatmap');

In [ ]:
df_train_tmp[ext_data_corrs_index].head()

In [ ]:
df_train_tmp[ext_data_corrs_index]
plot(df_train_tmp[ext_data_corrs_index])

In [ ]:
ext_data_corrs_index

In [ ]:
df_cp = df_train_tmp[ext_data_corrs_index]

df_cp['other'] = df_cp['other'].clip(0, 0.3*(10**7))


plot(df_cp[['other','poi']])



In [ ]:
df_cp['total_payments'] = df_cp['total_payments'].clip(0, 0.2*(10**8))


plot(df_cp[['total_payments','poi']])